In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import pandas.testing

## 1. Read in and view data

In [13]:
df_icu = pd.read_csv('data/Part1/ICUSTAYS.csv')
len(df_icu)

61532

the main useful columns in the ICUSTAYS table to us are: SUBJECT_ID, HADM_ID, (possibly) ICUSTAY_ID, and LOS.

I will also use INTIME and OUTTIME to make sure that the LOS is correct, especially since the database sources can be different

In [15]:
# convert date time format
df_icu.OUTTIME = pd.to_datetime(df_icu.OUTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_icu.INTIME = pd.to_datetime(df_icu.INTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

# recalculate LOS using difference between INTIME and OUTTIME to check the LOS column is correct
los_check = (df_icu.OUTTIME - df_icu.INTIME).apply(lambda x: (x.total_seconds()/86400))

# round both so they would match
df_icu['los_check'] = los_check.round(4)
df_icu['LOS_rounded'] = df_icu.LOS.round(4)


In [ ]:
# check that my calculation of ICU LOS and the one given match:
pandas.testing.assert_series_equal(df_icu.los_check, df_icu.LOS_rounded)

so the LOS that is given is not always equal to the difference between outtime and intime - I will investigate this a bit further.

In [30]:
# check what the difference between these are
df_icu['LOS_diff'] = abs(df_icu.LOS_rounded - df_icu.los_check)

df_icu['LOS_equals'] = np.where(abs(df_icu.los_check-df_icu.LOS_rounded)==0, 1, 0)

df_icu.loc[df_icu['LOS_equals']==0][['LOS_diff']]

,LOS_diff
61,0.0001
869,0.0001
963,0.0001
1033,NaN
1067,0.0001
...,...
59055,0.0001
59705,0.0001
60576,0.0001
61122,0.0001


it turns out most of the differences was due to rounding differences so I will look for places where differences are greater than 0.0002 (for some reason, differences = to 0.0001 get flagged even with <=)

In [29]:
df_icu['LOS_equals'] = np.where(abs(df_icu.los_check-df_icu.LOS_rounded)<=0.0002, 1, 0)

df_icu.loc[df_icu['LOS_equals'] == 0][['LOS_diff','OUTTIME']]

,LOS_diff,OUTTIME
1033,NaN,NaT
6451,NaN,NaT
10165,NaN,NaT
10507,NaN,NaT
13080,NaN,NaT
15156,NaN,NaT
17759,NaN,NaT
18442,NaN,NaT
27840,NaN,NaT
29117,NaN,NaT


so it turns out the only rows that are causing this is due to NaT in the OUTTIME and having a NaN in the LOS. Good, because we want to remove these anyway! Let's go ahead and remove any rows that have NaT for INTIME, OUTTIME, and LOS.

In [6]:
df_icu.dropna(subset = ["LOS"], inplace=True)
df_icu.dropna(subset = ["INTIME"], inplace=True)
df_icu.dropna(subset = ["OUTTIME"], inplace=True)

In [7]:
len(df_icu)

61522

looks like 10 cases had LOS=NaN (there weren't any more that had INTIME = NaT or OUTTIME = NaT)

In [8]:
# take out the columns I want to keep
df_icu = df_icu[["SUBJECT_ID", "HADM_ID","ICUSTAY_ID","LOS", "OUTTIME"]]

In [10]:
print('max ICU stay: %f' % max(df_icu.LOS))
print('min ICU stay: %f' % min(df_icu.LOS))

max ICU stay: 173.072500
min ICU stay: 0.000100


### ADMISSIONS DATA

Use this for multiple admits - # of admissions can be used as a predictor

I am avoiding using discharge location (for example, home) because that is something that would not be known for future patients until they are about to be discharged

In [ ]:
df_adm = pd.read_csv('data/ADMISSIONS.csv')